# TFS API呼叫測試

In [4]:
import tfs_api_helper as th
import pandas as pd

In [6]:
import urllib3
urllib3.disable_warnings()
tfs = th.TfsHelper(url='https://tfsa0w0p02:8080/tfs/', collection='C004', user='ESB11964', token='lfj2ve7ttgozouyvvb3azmlvgfvs7m47gbxmqfuw3ackoo6kzoca')
tfs_projects = tfs.list_projects().json()
tfs_projects

{'count': 53,
 'value': [{'id': '511e9787-2e5a-4913-8bf0-fa85ed62fe03',
   'name': 'DE-PL-Dh-Online-Loan',
   'description': '線上核貸改版',
   'url': 'https://tfsa0w0p02:8080/tfs/C004/_apis/projects/511e9787-2e5a-4913-8bf0-fa85ed62fe03',
   'state': 'wellFormed',
   'revision': 26967,
   'visibility': 'private'},
  {'id': '7a52afca-9494-4c3c-82d8-64b3b0eb8136',
   'name': 'DA-WM-Auto-Rebalance',
   'description': '理財Auto-Rebalance',
   'url': 'https://tfsa0w0p02:8080/tfs/C004/_apis/projects/7a52afca-9494-4c3c-82d8-64b3b0eb8136',
   'state': 'wellFormed',
   'revision': 26958,
   'visibility': 'private'},
  {'id': '2815c30b-0562-477c-8978-a62699a70376',
   'name': 'RI-PL-upl-acard-recalibration',
   'description': '純信進件評分卡調整',
   'url': 'https://tfsa0w0p02:8080/tfs/C004/_apis/projects/2815c30b-0562-477c-8978-a62699a70376',
   'state': 'wellFormed',
   'revision': 26813,
   'visibility': 'private'},
  {'id': '94d56edd-6d3f-4840-ae4c-77518eb3c71e',
   'name': 'DA-ESUN-Ofsn',
   'description': 

查詢專案內所有的WIT

In [ ]:
wiql = {'query': 'Select [System.ID] From WorkItems Where [System.TeamProject] = "C010-Tfs-Workitem"'}
tfs_list_workitems = tfs.list_workitems(wiql).json()

print(tfs_list_workitems)

查詢今天之前的WIT

In [ ]:
wiql = {'query': 'Select [System.ID] From WorkItems Where [System.TeamProject] = "C010-Tfs-Workitem" AND [Microsoft.VSTS.Scheduling.FinishDate] < @Today'}     
tfs_list_workitems = tfs.list_workitems(wiql).json()['workItems']

print(len(tfs_list_workitems))

查詢逾期的WIT(作用中或已提議)

In [ ]:
wiql = {'query': 'Select [System.ID] From WorkItems Where [System.TeamProject] = "C010-Tfs-Workitem" AND [Microsoft.VSTS.Scheduling.FinishDate] < @Today AND ([System.State] = "已提議" OR [System.State] = "作用中" )'}     
tfs_list_workitems = tfs.list_workitems(wiql).json()['workItems']

print(len(tfs_list_workitems))

查詢需要幫忙的工作項

In [ ]:
wiql = {'query': 'Select [System.ID] From WorkItems Where [System.TeamProject] = "C010-Tfs-Workitem" AND [C005.CallHelp] = true'}     
tfs_list_workitems = tfs.list_workitems(wiql).json()['workItems']

print(len(tfs_list_workitems))

查詢可能逾期的工作項

In [ ]:
wiql = {'query': 'Select [System.ID] From WorkItems Where [System.TeamProject] = "C010-Tfs-Workitem" AND [C005.OnSchedule] = false AND [System.WorkItemType] = "工作" AND ([System.State] = "已提議" OR [System.State] = "作用中" )'}     
tfs_list_workitems = tfs.list_workitems(wiql).json()['workItems']

print(len(tfs_list_workitems))

取出工作項的資料

In [ ]:
#使用正規表示式取員編
def get_staff_id(assigned):
    pattern = r'[0-9]{1,5}>'
    find = re.findall(string=assigned, pattern=pattern)[0]
    pattern = r'>'
    
    staff_id = re.split(pattern=pattern, string= find)
    return staff_id[0]

In [ ]:
import re

for i in tfs_list_workitems:
    workitem = tfs.tfs_api_call(i['url']).json()
    print(workitem)
    project = workitem['fields']['System.TeamProject']
    assigned = workitem['fields']['System.AssignedTo']
    title =  workitem['fields']['System.Title']
    html = workitem['_links']['html']['href']
    print('Project：' , project)
    print('工作包擁有者：', assigned)
    print('ID：', i['id'])
    print('工作項：', title)
    print('連結：', html)
    #取AD
    #pattern =  r'OANT\\[a-zA-Z0-9]*'
    staff = get_staff_id(assigned)
    print('Staff：', staff)
    
    
    

取得Mail Address

In [ ]:
mail_lists = pd.read_csv('./email_rank.csv', encoding = 'utf-8', skipinitialspace = True) #去空白
mail_lists

In [ ]:
mail_lists.columns
#mail_lists.to_csv('./email_rank2.csv', index = False)

In [ ]:

fliter = (mail_lists['PID'] == 11965)
mail_address = mail_lists[fliter]['EML'].values[0]

print(mail_address)

In [ ]:
type(mail_address)
mail_address

In [ ]:
#序列化 反序列化測試
import json
json.loads(r.text)
json.dumps(r_json)


**建立TFS服務** 測試差異

In [ ]:
import tfs_api_helper as th
import pandas as pd

tfs = th.TfsHelper('ESB11964','lfj2ve7ttgozouyvvb3azmlvgfvs7m47gbxmqfuw3ackoo6kzoca')
tfs_projects = tfs.get_project('C004').json()['value']

df = pd.DataFrame(tfs_projects)
df

In [ ]:
import tfs_api_helper as th

import pandas as pd

tfs = th.TfsHelper(url='https://tfsa0w0p02:8080/tfs/', collection='C004', user='ESB11964', token='lfj2ve7ttgozouyvvb3azmlvgfvs7m47gbxmqfuw3ackoo6kzoca')
#tfs_projects = tfs.list_projects().json()

In [ ]:
witql = {'query': 'Select * From WorkItems Where [C005.OnSchedule] = true'}

wit_result = tfs.list_workitems(witql).json()['workItems']
df = pd.DataFrame(wit_result)
df

In [ ]:
for index, row in df.iterrows():
   print(row['url'])

寄送Mail

In [ ]:
import esun_mail_helper as ml

In [ ]:
mail = ml.EsunMailHelper(sender= 'poting.yeh@gmail.com', host= '172.17.232.39', port=25)


In [ ]:
mail.send(receivers = ['ybt-11964@email.esunbank.com.tw'],  subject= '201812061337測試信件', message= '這邊是本文')
